In [1]:
!pip install open_clip_torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 41.3 kB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing instal

In [2]:
import torch
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [3]:
import open_clip
from open_clip import tokenize
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from tqdm import tqdm
import json

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model, _, preprocess = open_clip.create_model_and_transforms('ViT-L-14', pretrained='openai')
model.to(device)

model_path = "/kaggle/input/clip_model/pytorch/default/1/clip_model.pt"
checkpoints = torch.load(model_path, map_location=device)
model.load_state_dict(checkpoints['model_state_dict'])


open_clip_model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/open_clip/factory.py:388: UserWarning: These pretrained weights were trained with QuickGELU activation but the model config does not have that enabled. Consider using a model config with a "-quickgelu" suffix or enable with a flag.
  warnings.warn(


<All keys matched successfully>

In [5]:
for param in model.parameters():
    param.requires_grad = False

In [6]:

for name, param in model.named_parameters():
    print(f"{name} norm: {param.norm().item()}")

positional_embedding norm: 2.0288569927215576
text_projection norm: 11.265134811401367
logit_scale norm: 4.605087757110596
visual.class_embedding norm: 5.109840393066406
visual.positional_embedding norm: 11.497537612915039
visual.proj norm: 12.942944526672363
visual.conv1.weight norm: 13.028203010559082
visual.ln_pre.weight norm: 21.016136169433594
visual.ln_pre.bias norm: 2.6796717643737793
visual.transformer.resblocks.0.ln_1.weight norm: 14.776494026184082
visual.transformer.resblocks.0.ln_1.bias norm: 3.969780683517456
visual.transformer.resblocks.0.attn.in_proj_weight norm: 17.87546157836914
visual.transformer.resblocks.0.attn.in_proj_bias norm: 35.37779235839844
visual.transformer.resblocks.0.attn.out_proj.weight norm: 12.688957214355469
visual.transformer.resblocks.0.attn.out_proj.bias norm: 2.594679355621338
visual.transformer.resblocks.0.ln_2.weight norm: 32.28483581542969
visual.transformer.resblocks.0.ln_2.bias norm: 4.974412441253662
visual.transformer.resblocks.0.mlp.c_fc.w

In [7]:
json_path="/kaggle/input/data-info1/output.json"
image_path="/kaggle/input/images/images/"

In [8]:
with open(json_path,"r",encoding="utf-8") as f:
    data=json.load(f)

image_list = []
title_list = []

for i in data:
    image_list.append(image_path + i['image'])
    title_list.append(i['caption'])
class MyDataset(Dataset):
    def __init__(self, list_image_path, list_txt):
        self.image_path = list_image_path
        self.title = tokenize(list_txt)
    def __len__(self):
        return len(self.image_path)
    def __getitem__(self, idx):
        image = preprocess(Image.open(self.image_path[idx]).convert('RGB'))
        title = self.title[idx]
        return image, title


dataset=MyDataset(image_list,title_list)
dataloader=DataLoader(dataset, batch_size=4, shuffle=False)

In [9]:

class CrossAttention(nn.Module):
    def __init__(self, embed_dim, num_heads, dropout=0.0):
        super().__init__()
        self.multihead_attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(dropout)
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, 4 * embed_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(4 * embed_dim, embed_dim),
            nn.Dropout(dropout)
        )
        
    def forward(self, q, k, v):
        attn_output, _ = self.multihead_attn(
            q.transpose(0, 1),
            k.transpose(0, 1),
            v.transpose(0, 1)
        )
        attn_output = attn_output.transpose(0, 1)
        q = q + self.dropout(attn_output)
        q = self.norm1(q)
        
        q2 = self.mlp(q)
        q = q + self.dropout(q2)
        q = self.norm2(q)
        
        return q

class CLIPCrossAttentionWrapper(nn.Module):
    def __init__(self, clip_model, embed_dim=768, num_heads=8, dropout=0.1):
        super().__init__()
        self.clip_model = clip_model
        self.clip_embed_dim = embed_dim  
        
        self.img_to_text_attn = CrossAttention(embed_dim, num_heads, dropout)
        
        self.text_to_img_attn = CrossAttention(embed_dim, num_heads, dropout)
        
        self.img_proj = nn.Linear(self.clip_model.visual.output_dim, embed_dim)
        self.text_proj = nn.Linear(self.clip_model.transformer.resblocks[-1].attn.out_proj.out_features, embed_dim)
        
        self.final_img_proj = nn.Linear(embed_dim, self.clip_model.visual.output_dim)
        self.final_text_proj = nn.Linear(embed_dim, self.clip_model.transformer.resblocks[-1].attn.out_proj.out_features)
        
        self.logit_scale = self.clip_model.logit_scale
        
    def forward(self, images, texts):
        image_features = self.clip_model.encode_image(images)
        text_features = self.clip_model.encode_text(texts)
        
        img_features_proj = self.img_proj(image_features.unsqueeze(1))  
        text_features_proj = self.text_proj(text_features.unsqueeze(1))  
        
        img_attended = self.img_to_text_attn(img_features_proj, text_features_proj, text_features_proj)
        text_attended = self.text_to_img_attn(text_features_proj, img_features_proj, img_features_proj)
        
        img_features_final = self.final_img_proj(img_attended.squeeze(1))
        text_features_final = self.final_text_proj(text_attended.squeeze(1))
        

        image_features = F.normalize(img_features_final, dim=-1)
        text_features = F.normalize(text_features_final, dim=-1)
        
        return image_features, text_features, self.logit_scale

def clip_loss(logits_per_image, logits_per_text):
    ground_truth = torch.arange(len(logits_per_image), dtype=torch.long, device=device)
    loss_i = F.cross_entropy(logits_per_image, ground_truth)
    loss_t = F.cross_entropy(logits_per_text, ground_truth)
    return (loss_i + loss_t) / 2

In [10]:
clip_model = CLIPCrossAttentionWrapper(model).to(device)
clip_model

CLIPCrossAttentionWrapper(
  (clip_model): CLIP(
    (visual): VisionTransformer(
      (conv1): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
      (patch_dropout): Identity()
      (ln_pre): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (transformer): Transformer(
        (resblocks): ModuleList(
          (0-23): 24 x ResidualAttentionBlock(
            (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
            )
            (ls_1): Identity()
            (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (mlp): Sequential(
              (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
              (gelu): GELU(approximate='none')
              (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
            )
            (ls_2): Identity

In [11]:

optimizer = torch.optim.AdamW([
    {'params':clip_model.img_to_text_attn.parameters()},
    {'params':clip_model.text_to_img_attn.parameters()},
    {'params':clip_model.img_proj.parameters()},
    {'params':clip_model.text_proj.parameters()},
    {'params':clip_model.final_img_proj.parameters()},
    {'params':clip_model.final_text_proj.parameters()}
], lr=1e-6)

In [12]:
def train(dataloader, model, optimizer, save_path=""):
    train_loader = tqdm(dataloader, total=len(dataloader))
    num = 0
    best_loss = float('inf')

    for batch in train_loader:
        num += 1
        optimizer.zero_grad()

        images, texts = batch
        images = images.to(device)
        texts = texts.to(device)

        image_features, text_features, logit_scale = model(images, texts)
        
        logits_per_image = logit_scale * image_features @ text_features.t()
        logits_per_text = logits_per_image.t()

        loss = clip_loss(logits_per_image, logits_per_text)

        loss.backward()
        optimizer.step()

        if num % 100 == 0:
            print(f"Step {num}, total_loss: {loss.item()}")
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'step': num,
                'loss': best_loss
            }, save_path)
            
        if loss.item() < best_loss:
            print(f"New best loss: {loss.item()} (previous: {best_loss})")
            best_loss = loss.item()

train(dataloader, clip_model, optimizer, save_path="/kaggle/working/clip_cross_attention_model.pt")    

  0%|          | 1/16359 [00:01<7:52:48,  1.73s/it]

New best loss: 1.3757402896881104 (previous: inf)


  0%|          | 4/16359 [00:03<3:22:31,  1.35it/s]

New best loss: 1.3747957944869995 (previous: 1.3757402896881104)


  0%|          | 6/16359 [00:04<2:58:01,  1.53it/s]

...

 99%|█████████▉| 16199/16359 [3:43:25<02:05,  1.27it/s]

Step 16200, total_loss: 0.26308974623680115


100%|█████████▉| 16299/16359 [3:44:50<00:48,  1.25it/s]

Step 16300, total_loss: 0.15562966465950012


100%|██████████| 16359/16359 [3:45:44<00:00,  1.21it/s]


In [13]:

for name, param in clip_model.named_parameters():
    print(f"{name} norm: {param.norm().item()}")

logit_scale norm: 4.605087757110596
clip_model.positional_embedding norm: 2.0288569927215576
clip_model.text_projection norm: 11.265134811401367
clip_model.visual.class_embedding norm: 5.109840393066406
clip_model.visual.positional_embedding norm: 11.497537612915039
clip_model.visual.proj norm: 12.942944526672363
clip_model.visual.conv1.weight norm: 13.028203010559082
clip_model.visual.ln_pre.weight norm: 21.016136169433594
clip_model.visual.ln_pre.bias norm: 2.6796717643737793
clip_model.visual.transformer.resblocks.0.ln_1.weight norm: 14.776494026184082
clip_model.visual.transformer.resblocks.0.ln_1.bias norm: 3.969780683517456
clip_model.visual.transformer.resblocks.0.attn.in_proj_weight norm: 17.87546157836914
clip_model.visual.transformer.resblocks.0.attn.in_proj_bias norm: 35.37779235839844
clip_model.visual.transformer.resblocks.0.attn.out_proj.weight norm: 12.688957214355469
clip_model.visual.transformer.resblocks.0.attn.out_proj.bias norm: 2.594679355621338
clip_model.visual.t